In [ ]:
from mylibraries import *
X_train = pd.read_csv('X_train_scaled.csv')
X_test = pd.read_csv('X_test_scaled.csv')
y_train = pd.read_csv('y_train.csv')
y_test = pd.read_csv('y_test.csv')

y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

# Získanie názvov stĺpcov z pôvodného X_train
column_names = X_train.columns

In [ ]:
# Definovanie modelu DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(max_depth=7, random_state=42)

# Trénovanie modelu
dtc.fit(X_train, y_train)

# Predikcia na testovacej množine
y_pred = dtc.predict(X_test)

# Vyhodnotenie modelu
accuracy = accuracy_score(y_test, y_pred)
print(f"Testovacia presnosť: {accuracy:.4f}")

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score

# Výpis klasifikačnej správy
print("Klasifikačná správa:")
print(classification_report(y_test, y_pred))

# Matica zámen
conf_matrix = confusion_matrix(y_test, y_pred)
print("Matica zámen:")
print(conf_matrix)

# ROC krivka
y_pred_proba = dtc.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
plt.plot(fpr, tpr, label='ROC krivka')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel('Falošne pozitívna miera')
plt.ylabel('Pravdivo pozitívna miera')
plt.title('ROC Krivka')
plt.legend()
plt.show()

from sklearn.metrics import precision_recall_curve, average_precision_score

# Precision-Recall krivka
precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
average_precision = average_precision_score(y_test, y_pred_proba)

plt.figure()
plt.step(recall, precision, where='post', color='b', alpha=0.2, label=f'Priemerná presnosť = {average_precision:.2f}')
plt.fill_between(recall, precision, step='post', alpha=0.2, color='b')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('Precision-Recall Krivka')
plt.legend(loc="lower left")
plt.show()


from sklearn.model_selection import learning_curve

# Learning Curve
train_sizes, train_scores, test_scores = learning_curve(dtc, X_train, y_train, cv=5, n_jobs=-1, train_sizes=np.linspace(0.1, 1.0, 10))
train_scores_mean = np.mean(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)

plt.figure()
plt.plot(train_sizes, train_scores_mean, 'o-', color='r', label='Trénovacia presnosť')
plt.plot(train_sizes, test_scores_mean, 'o-', color='g', label='Validačná presnosť')

plt.xlabel('Počet trénovacích príkladov')
plt.ylabel('Presnosť')
plt.title('Learning Curve')
plt.legend(loc="best")
plt.show()

# Dôležitosť premenných
importances = dtc.feature_importances_
feature_names = column_names
feature_importances = pd.Series(importances, index=feature_names)

# Zobrazenie dôležitosti premenných
feature_importances.nlargest(10).plot(kind='barh')
plt.title('Dôležitosť premenných')
plt.show()


In [ ]:
## Výber len najdôležitejších atribútov
important_features = feature_importances.nlargest(10).index.tolist()  # Konvertovanie Indexu na list
important_feature_indices = [column_names.get_loc(feature) for feature in important_features]  # Indexy najdôležitejších atribútov

X_train_important = X_train.iloc[:, important_feature_indices]
X_test_important = X_test.iloc[:, important_feature_indices]

# Definovanie modelu DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
dtc_important = DecisionTreeClassifier(max_depth=7, random_state=42)

# Trénovanie modelu len na najdôležitejších atribútoch
dtc_important.fit(X_train_important, y_train)

# Predikcia na testovacej množine
y_pred_important = dtc_important.predict(X_test_important)

# Vyhodnotenie modelu
accuracy_important = accuracy_score(y_test, y_pred_important)
print(f"Testovacia presnosť s najdôležitejšími atribútmi: {accuracy_important:.4f}")

# Klasifikačná správa
print("Klasifikačná správa s najdôležitejšími atribútmi:")
print(classification_report(y_test, y_pred_important))

# Matica zámen
conf_matrix_important = confusion_matrix(y_test, y_pred_important)
print("Matica zámen s najdôležitejšími atribútmi:")
print(conf_matrix_important)


In [ ]:
from sklearn.ensemble import BaggingClassifier

# Definovanie modelu Decision Tree Classifier
dtc = DecisionTreeClassifier(max_depth=7, random_state=42)

# Definovanie modelu Bagging Classifier
bagging = BaggingClassifier(estimator=dtc, n_estimators=10, random_state=42)

# Trénovanie modelu Bagging
bagging.fit(X_train, y_train)

# Predikcia na testovacej množine
y_pred_bagging = bagging.predict(X_test)

# Vyhodnotenie modelu Bagging
accuracy_bagging = accuracy_score(y_test, y_pred_bagging)
print(f"Testovacia presnosť Bagging modelu: {accuracy_bagging:.4f}")

# Klasifikačná správa Bagging modelu
print("Klasifikačná správa Bagging modelu:")
print(classification_report(y_test, y_pred_bagging))

# Matica zámen Bagging modelu
conf_matrix_bagging = confusion_matrix(y_test, y_pred_bagging)
print("Matica zámen Bagging modelu:")
print(conf_matrix_bagging)